# Farina in Natura - estratégia

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import datetime
import fpdf as FPDF


In [13]:
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [14]:
# para resolver o problema: "(unicode error) 'utf-8' codec can't decode "
# https://www.youtube.com/watch?v=ZSC7X4N8RKo,
#Erro: 
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 900: invalid continuation byte
# enem = pd.read_csv("MICRODADOS_ENEM_2020.csv", encoding="utf-8", sep = ';')
# enem = pd.read_csv("MICRODADOS_ENEM_2020.csv", encoding="ISO-8859-1", sep = ';')

transactions = pd.read_csv(r'C:\Users\ctobr\OneDrive\DATA_Science\1_farina_in_natura\farina_in_natura00\CSV\farina.CSV', 
encoding="ISO-8859-1", sep = ';')

In [15]:
transactions.head()

,venda,data_hora,cliente,mesa,item,produto,quantidade,valor_unit,desconto,total_item
0,460.0,01/02/2022 08:13,CONSUMIDOR,M 1,119.0,SUCO DE LARANJA 400ML,1.0,"14,5",0.0,"14,5"
1,460.0,01/02/2022 08:13,CONSUMIDOR,M 1,121.0,PAO SOURDOUGH OVOS,1.0,"15,5",0.0,"15,5"
2,461.0,01/02/2022 08:14,CONSUMIDOR,M 2,119.0,SUCO DE LARANJA 400ML,1.0,"14,5",0.0,"14,5"
3,461.0,01/02/2022 08:14,CONSUMIDOR,M 2,121.0,PAO SOURDOUGH OVOS,1.0,"15,5",0.0,"15,5"
4,465.0,01/02/2022 09:01,CONSUMIDOR,M 1,46.0,QUICHE BACON 90G,2.0,"19,5",0.0,39


In [16]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25998 entries, 0 to 25997
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   venda       25992 non-null  float64
 1   data_hora   25992 non-null  object 
 2   cliente     25992 non-null  object 
 3   mesa        22779 non-null  object 
 4   item        25922 non-null  float64
 5   produto     25992 non-null  object 
 6   quantidade  25992 non-null  float64
 7   valor_unit  25992 non-null  object 
 8   desconto    25992 non-null  float64
 9   total_item  25992 non-null  object 
dtypes: float64(4), object(6)
memory usage: 2.0+ MB


# Limpeza e manipulação dos dados

In [17]:
### CHECK OUT IF THE NUMBER OF TOTAL ENTRIES IS EQUAL TO THE NUMBER OF ENTRIES OF THE VARIABLES  

transactions.isnull().sum()

venda            6
data_hora        6
cliente          6
mesa          3219
item            76
produto          6
quantidade       6
valor_unit       6
desconto         6
total_item       6
dtype: int64

In [18]:
## DROPPING THE ROWS THAT ALL COLUMN VALUES ARE NULL
#  https://www.youtube.com/watch?v=uDr67HBIPz8&t=818s

transactions = transactions.dropna(how='all')

In [19]:
# Delete the column "item"

del transactions["item"]

In [20]:
transactions.info()
transactions.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25992 entries, 0 to 25997
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   venda       25992 non-null  float64
 1   data_hora   25992 non-null  object 
 2   cliente     25992 non-null  object 
 3   mesa        22779 non-null  object 
 4   produto     25992 non-null  object 
 5   quantidade  25992 non-null  float64
 6   valor_unit  25992 non-null  object 
 7   desconto    25992 non-null  float64
 8   total_item  25992 non-null  object 
dtypes: float64(3), object(6)
memory usage: 2.0+ MB


,venda,data_hora,cliente,mesa,produto,quantidade,valor_unit,desconto,total_item
0,460.0,01/02/2022 08:13,CONSUMIDOR,M 1,SUCO DE LARANJA 400ML,1.0,"14,5",0.0,"14,5"
1,460.0,01/02/2022 08:13,CONSUMIDOR,M 1,PAO SOURDOUGH OVOS,1.0,"15,5",0.0,"15,5"
2,461.0,01/02/2022 08:14,CONSUMIDOR,M 2,SUCO DE LARANJA 400ML,1.0,"14,5",0.0,"14,5"
3,461.0,01/02/2022 08:14,CONSUMIDOR,M 2,PAO SOURDOUGH OVOS,1.0,"15,5",0.0,"15,5"
4,465.0,01/02/2022 09:01,CONSUMIDOR,M 1,QUICHE BACON 90G,2.0,"19,5",0.0,39


In [21]:
## Using isna() to select all rows with NaN in the column ('mesa') and the value "CONSUMIDOR" on the column "cliente" 
## and replace the "mesa" value with "M0"
## https://datatofish.com/rows-with-nan-pandas-dataframe/
## https://www.geeksforgeeks.org/how-to-replace-values-in-column-based-on-condition-in-pandas/

transactions.loc[(transactions["cliente"] == 'CONSUMIDOR') &  (transactions['mesa'].isna()), 'mesa']= 'M0'


In [22]:
# Checking if the changes were correct.

transactions.loc[transactions["mesa"] == 'M0']

,venda,data_hora,cliente,mesa,produto,quantidade,valor_unit,desconto,total_item
121,520.0,02/02/2022 16:57,CONSUMIDOR,M0,BOMBOLONI NUTELLA 80G,2.0,12,0.0,24
122,520.0,02/02/2022 16:57,CONSUMIDOR,M0,TARTELETE DE MORANGO 120G,1.0,18,0.0,18
123,520.0,02/02/2022 16:57,CONSUMIDOR,M0,CROISSANT MUSSATRELA DE,1.0,"25,5",0.0,"25,5"
894,856.0,11/02/2022 19:19,CONSUMIDOR,M0,CROISSANT BRIE PARMA MEL DE,2.0,"25,5",0.0,51
895,856.0,11/02/2022 19:19,CONSUMIDOR,M0,VH SANTA HORTENSIA SAUVIG,1.0,59,0.0,59


In [23]:
# Assigning a value "ND" to the empty records in column "mesa"
# https://www.w3schools.com/python/pandas/pandas_cleaning_empty_cells.asp

transactions['mesa'].fillna("ND",inplace = True )

In [24]:
# Find Unique Values in the Column "produto"
# https://www.statology.org/pandas-unique-values-in-column/
transactions.produto.value_counts()

CROISSANT                         2393
PASTEL DE NATA 60G                1339
CROISSANT  AMENDOAS 100G           742
FOLHADO MACA 80G                   719
SANDUICHE DE PEPERONI BRIE         596
COOKIES 80G                        559
CROISSANT IFOOD                    534
SOURDOUGH AZEITONA E ALECRIM       517
FOLHADO FRANGO 160G                500
BOMBOLONI DOCE DE LEITE 80G        487
FOCACCIA FORMAGGIO 246G            460
AGUA MINERAL S/GAS 500ML           446
TORRADA FARINA 165G                435
CAFE EQUILIBRADO CAPSULA P         431
SOURDOUGH 369G                     431
LARANJA 400 ML                     430
MINI CROISSANT 25G KIT C 5         427
CROISSANT BRIE PARMA MEL DE        423
PIZZA AL TAGLIO GORGONZOLA E       421
MINI PAIN AU CHOCOLAT 35G          419
BOMBOLONI NUTELLA 80G              409
CROISSANT  TRADICIONAL 80 G        396
QUICHE DE FRANGO ALHO PORO         360
AGUA MINERAL SEM GAS 500 ML        341
COCA COLA LATA 350 ML              337
SUCO DE LARANJA 400ML    

In [25]:
## Get unique names in the "produto" column using Series.unique() 
## https://sparkbyexamples.com/pandas/pandas-count-unique-values-in-column/
transactions.produto.unique().size

212

In [26]:
# Convert the "venda" variable from float64 type to integer type
# https://stackoverflow.com/questions/43956335/convert-float64-column-to-int64-in-pandas

transactions['venda'] = np.int64(transactions['venda'])

In [27]:
# Convert "data" to datetime object
transactions['data_hora'] = pd.to_datetime(transactions['data_hora'], format='%d/%m/%Y %H:%M')

#How to Solve Python ValueError: unconverted data remains
#read://https_researchdatapod.com/?url=https%3A%2F%2Fresearchdatapod.com%2Fhow-to-solve-python-valueerror-unconverted-data-remains%2F

In [28]:
# Convert the "quantidade" variable from float64 type to integer type
# https://stackoverflow.com/questions/43956335/convert-float64-column-to-int64-in-pandas

transactions['quantidade'] = np.int64(transactions['quantidade'])

In [29]:
# change the decimal delimiter of the the variable "valor_unit" from "," to "."
# https://cursos.alura.com.br/forum/topico-transformar-pontos-por-virgulas-dentro-de-um-dataframe-121090
transactions['valor_unit'] = transactions['valor_unit'].str.replace(",",".")

# Convert the "valor_uni" variable from object type to float64 type
# https://www.statology.org/pandas-convert-object-to-float/

transactions['valor_unit'] = transactions['valor_unit'].astype(float)

In [30]:
# change the decimal delimiter of the the variable "total_item" from "," to "."
# https://cursos.alura.com.br/forum/topico-transformar-pontos-por-virgulas-dentro-de-um-dataframe-121090
transactions['total_item'] = transactions['total_item'].str.replace(",",".")

# Convert the "total_item" variable from object type to float64 type
# https://www.statology.org/pandas-convert-object-to-float/

transactions['total_item'] = transactions['total_item'].astype(float)

In [31]:
# Verificando se o valor "total_item" é igual a: "quantidade" * "valor_unit"

transactions["total_item_verif"] = transactions['quantidade']* transactions['valor_unit']

In [32]:
# Continuando o item anterior
# https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/

transactions['confirm'] = np.where(transactions['total_item'] > transactions['total_item_verif'], True, False)


In [33]:
# continuando o item anterior
transactions.loc[transactions["confirm"] == True].sum()

C:\Users\ctobr\AppData\Local\Temp\ipykernel_42736\3180257399.py:2: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  transactions.loc[transactions["confirm"] == True].sum()


venda                                                        13730391
cliente             CONSUMIDORCONSUMIDORCONSUMIDORCONSUMIDORIFOODC...
mesa                M 5M 13M 11M 5NDM 19NDM 13M 13M 17NDM 23M 23M ...
produto             CROISSANTCROISSANTCROISSANTCROISSANTCROISSANT ...
quantidade                                                       2139
valor_unit                                                    21083.7
desconto                                                          0.0
total_item                                                    53878.3
total_item_verif                                              22219.1
confirm                                                          2032
dtype: object

In [34]:
# How do I select specific rows and columns from a DataFrame?
# https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html#how-do-i-select-specific-rows-and-columns-from-a-dataframe
# https://www.geeksforgeeks.org/select-rows-columns-by-name-or-index-in-pandas-dataframe-using-loc-iloc/

ts2 = transactions.loc[transactions["confirm"] == True, ["venda", "produto", "quantidade", "valor_unit", "total_item", "total_item_verif"]]

In [35]:
ts2

,venda,produto,quantidade,valor_unit,total_item,total_item_verif
7039,3571,CROISSANT,1,10.0,25.5,10.0
7047,3575,CROISSANT,1,10.0,28.9,10.0
7054,3582,CROISSANT,1,10.0,25.5,10.0
7072,3599,CROISSANT,1,10.0,19.5,10.0
7074,3600,CROISSANT IFOOD,1,12.0,51.0,12.0
7086,3603,CROISSANT,1,10.0,19.5,10.0
7088,3606,CROISSANT IFOOD,1,12.0,29.5,12.0
7110,3616,CROISSANT,1,10.0,25.5,10.0
7111,3616,CROISSANT,1,10.0,28.9,10.0
7112,3619,CROISSANT,1,10.0,25.5,10.0


In [36]:
# Exporting to a pdf file the transactions that has a inconsistent data.
from matplotlib.backends.backend_pdf import PdfPages


In [70]:
# https://levelup.gitconnected.com/how-to-write-a-pandas-dataframe-as-a-pdf-5cdf7d525488

def _draw_as_table(df, pagesize):
    alternating_colors = [['white'] * len(df.columns), ['lightgray'] * len(df.columns)] * len(df)
    alternating_colors = alternating_colors[:len(df)]
    fig, ax = plt.subplots(figsize=pagesize)
    ax.axis("tight")
    ax.axis('off')
    the_table = ax.table(cellText=df.values,
                        rowLabels=df.index,
                        colLabels=df.columns,
                        rowColours=['lightblue']*len(df),
                        colColours=['lightblue']*len(df.columns),
                        cellColours=alternating_colors,
                        loc='center')
    return fig


In [ ]:
# https://www.alixaprodev.com/2022/05/convert-pandas-dataframe-to-pdf-in-python.html
#fig, ax =plt.subplots(figsize=(14,6))
#ax.axis('equal')
#ax.axis('off')
#the_table = ax.table(cellText=ts2.values,colLabels=ts2.columns,loc='center')




In [67]:
def dataframe_to_pdf(df, filename, numpages=(1, 1), pagesize=(11, 8.5)):
  with PdfPages(filename) as pdf:
    nh, nv = numpages
    rows_per_page = len(df) // nh
    cols_per_page = len(df.columns) // nv
    for i in range(0, nh):
        for j in range(0, nv):
            page = df.iloc[(i*rows_per_page):min((i+1)*rows_per_page, len(df)),
                           (j*cols_per_page):min((j+1)*cols_per_page, len(df.columns))]
            fig = _draw_as_table(page, pagesize)
            if nh > 1 or nv > 1:
                # Add a part/page number at bottom-center of page
                fig.text(0.5, 0.5/pagesize[0],
                         "Part-{}x{}: Page-{}".format(i+1, j+1, i*nv + j + 1),
                         ha='center', fontsize=8)
            pdf.savefig(fig, bbox_inches='tight')
            
            plt.close()

In [68]:
### Trying to Align the page number
# https://www.geeksforgeeks.org/matplotlib-figure-figure-text-in-python/
def dataframe_to_pdf(df, filename, numpages=(1, 1), pagesize=(11, 8.5)):
  with PdfPages(filename) as pdf:
    nh, nv = numpages
    rows_per_page = len(df) // nh
    cols_per_page = len(df.columns) // nv
    for i in range(0, nh):
        for j in range(0, nv):
            page = df.iloc[(i*rows_per_page):min((i+1)*rows_per_page, len(df)),
                           (j*cols_per_page):min((j+1)*cols_per_page, len(df.columns))]
            fig = _draw_as_table(page, pagesize)
            #if nh > 1 or nv > 1:
                # Add a part/page number at bottom-center of page
                #fig.text(0.5, 0.5/pagesize[0],
                       # "Part-{}x{}: Page-{}".format(i+1, j+1, i*nv + j + 1),
                       # verticalalignment ='bottom', 
                       # horizontalalignment ='right',
                       #transform = ax.transAxes,
                        #color ='green',
                        #fontsize = 5
                         #)
            pdf.savefig(fig, bbox_inches='tight')
            
            plt.close()

In [71]:
dataframe_to_pdf(ts2, "fool4.pdf", numpages=(12, 1), pagesize=(11.7, 8.3))

In [74]:
import pdfkit

In [76]:
f = open('exp.html','w')
a = ts2.to_html()
f.write(a)
f.close()

In [78]:
pdfkit.from_file('exp.html', 'fool4.pdf')

OSError: No wkhtmltopdf executable found: "b''"
If this file exists please check that this process can read it or you can pass path to it manually in method call, check README. Otherwise please install wkhtmltopdf - https://github.com/JazzCore/python-pdfkit/wiki/Installing-wkhtmltopdf